In [1]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [2]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    # r'D:\code\forecast_model\notebook\20250817_base',
    # r'D:\code\forecast_model\notebook\20250820_base',
    # r'D:\code\forecast_model\notebook\20250820_bigger',
    # r'D:\code\forecast_model\notebook\20250821_mlp_base2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline3',
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline2",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline3",
    r'D:\code\forecast_model\notebook\20250825_tcn_baseline',
    ]

In [3]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,test_dummy_mean_class_f1,train_mean_class_f1_enhanced_pct,val_mean_class_f1_enhanced_pct,test_best_mean_class_f1_enhanced_pct,test_final_mean_class_f1_enhanced_pct,each_epoch_cost,cost,test_mean_class_f1,mean_test_01_f1,mean_val_01_f1
0,20250825_tcn_baseline_P100_250_seed0_IDX0_final,20250825_tcn_baseline_P100_250_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.500088,99.964631,51.228334,67.977829,51.325759,0.02h,4.14h,0.749820,0.840038,0.756275
1,20250825_tcn_baseline_P100_250_seed0_IDX0_best,20250825_tcn_baseline_P100_250_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.500088,99.964631,51.228334,67.977829,51.325759,0.02h,4.14h,0.837834,0.840038,0.756275
2,20250825_tcn_baseline_P100_250_seed1_IDX2_final,20250825_tcn_baseline_P100_250_seed1_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.499744,100.102513,51.102723,58.351238,51.238745,0.02h,4.18h,0.750139,0.791351,0.755127
3,20250825_tcn_baseline_P100_250_seed1_IDX2_best,20250825_tcn_baseline_P100_250_seed1_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.499744,100.102513,51.102723,58.351238,51.238745,0.02h,4.18h,0.794100,0.791351,0.755127
4,20250825_tcn_baseline_P100_250_seed2_IDX4_final,20250825_tcn_baseline_P100_250_seed2_IDX4,train_data,no,200,128,8192,0.0384,0,0,...,0.500730,99.708613,50.292033,74.915281,51.871392,0.02h,4.33h,0.754387,0.875852,0.752556
5,20250825_tcn_baseline_P100_250_seed2_IDX4_best,20250825_tcn_baseline_P100_250_seed2_IDX4,train_data,no,200,128,8192,0.0384,0,0,...,0.500730,99.708613,50.292033,74.915281,51.871392,0.02h,4.33h,0.878154,0.875852,0.752556
6,20250825_tcn_baseline_P100_250_seed3_IDX6_final,20250825_tcn_baseline_P100_250_seed3_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.498866,100.454712,51.409140,77.283028,52.526299,0.02h,3.86h,0.755348,0.884405,0.755329
7,20250825_tcn_baseline_P100_250_seed3_IDX6_best,20250825_tcn_baseline_P100_250_seed3_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.498866,100.454712,51.409140,77.283028,52.526299,0.02h,3.86h,0.889929,0.884405,0.755329
8,20250825_tcn_baseline_P100_250_seed4_IDX8_final,20250825_tcn_baseline_P100_250_seed4_IDX8,train_data,no,200,128,8192,0.0384,0,0,...,0.499855,100.058102,52.035534,62.174743,52.856081,0.02h,4.21h,0.757245,0.810638,0.759957
9,20250825_tcn_baseline_P100_250_seed4_IDX8_best,20250825_tcn_baseline_P100_250_seed4_IDX8,train_data,no,200,128,8192,0.0384,0,0,...,0.499855,100.058102,52.035534,62.174743,52.856081,0.02h,4.21h,0.792591,0.810638,0.759957


In [4]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [5]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'label_train',
 'label_val',
 'label_test_final',
 'model',
 'grad_norm_best',
 'grad_norm',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',

In [6]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250825_tcn_baseline_P100_250_seed0_IDX0_final,1(44761)@1(44761)
1,20250825_tcn_baseline_P100_250_seed0_IDX0_best,1(44761)@1(44761)
2,20250825_tcn_baseline_P100_250_seed1_IDX2_final,1(44761)@1(44761)
3,20250825_tcn_baseline_P100_250_seed1_IDX2_best,1(44761)@1(44761)
4,20250825_tcn_baseline_P100_250_seed2_IDX4_final,1(44761)@1(44761)
5,20250825_tcn_baseline_P100_250_seed2_IDX4_best,1(44761)@1(44761)
6,20250825_tcn_baseline_P100_250_seed3_IDX6_final,1(44761)@1(44761)
7,20250825_tcn_baseline_P100_250_seed3_IDX6_best,1(44761)@1(44761)
8,20250825_tcn_baseline_P100_250_seed4_IDX8_final,1(44761)@1(44761)
9,20250825_tcn_baseline_P100_250_seed4_IDX8_best,1(44761)@1(44761)


In [7]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'test_final_f1']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))

# 只考察 final
_match_bool = data['train_title'].apply(lambda x: 'final' in x)
d = d.loc[_match_bool]

# # 根据 idx 区分类别
# # 用于未区分title的情况
# _cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
# _cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# title 已经区分
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values('train_f1',ascending=False)


,train_loss,train_f1,val_f1,test_final_f1,cost
train_title,,,,,
20250825_tcn_baseline_P100_250_final,0.000005,1.000000,0.756394,0.759930,4.144h
20250825_tcn_baseline_P100_300_final,0.001992,0.999316,0.478070,0.431994,4.886h


In [8]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]